In [7]:
import torch
import sentencepiece as spm
import torch.nn as nn

# ---------------- MODEL DEFINITION ----------------
class TinyTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, num_heads=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, nhead=num_heads, dim_feedforward=embed_dim * 4
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        emb = self.embedding(x)
        emb = emb.permute(1, 0, 2)
        encoded = self.encoder(emb)
        out = self.fc(encoded)
        return out.permute(1, 0, 2)

# ---------------- LOAD TOKENIZER ----------------
sp = spm.SentencePieceProcessor(model_file="tok/compatika_sp.model")
vocab_size = sp.get_piece_size()

# ---------------- LOAD MODEL ----------------
model = TinyTransformer(vocab_size)
model.load_state_dict(torch.load("compatika_v1alpha_scratch.pt", map_location="cpu"))
model.eval()

# ---------------- GENERATION FUNCTION ----------------
def generate_response(prompt, max_len=40):
    tokens = sp.encode(prompt, out_type=int)
    x = torch.tensor(tokens, dtype=torch.long).unsqueeze(0)
    for _ in range(max_len):
        with torch.no_grad():
            out = model(x)
        next_id = out[0, -1].argmax().item()
        tokens.append(next_id)
        if next_id == 3:  # EOS token
            break
        x = torch.tensor(tokens, dtype=torch.long).unsqueeze(0)
    return sp.decode(tokens)

# ---------------- TEST ON FEW PROMPTS ----------------
tests = [
    "USER: I'm really upset. My friend ignored my message.\nCOMPATIKA:",
    "USER: I feel anxious about tomorrow.\nCOMPATIKA:",
    "USER: I want to tell my partner I need more time together.\nCOMPATIKA:",
    "USER: I feel guilty because I blamed someone unfairly.\nCOMPATIKA:"
]

for t in tests:
    print("=" * 60)
    print("Prompt:", t)
    print("Compatika:", generate_response(t))
    print()

c:\Users\aman\Desktop\Compatika-V1-alpha-ai-model-in-2-4-MB-dataset-and-1m-param\env\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
C:\Users\aman\AppData\Local\Temp\ipykernel_7736\1014568064.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are expli

Prompt: USER: I'm really upset. My friend ignored my message.
COMPATIKA:
Compatika: USER: I'm really upset. My friend ignored my message. COMPATIKA: My friend'm really upset. I'm really upset. I'm really upset. I'm really upset. I'm really upset. I'm really upset. I'm

Prompt: USER: I feel anxious about tomorrow.
COMPATIKA:
Compatika: USER: I feel anxious about tomorrow. COMPATIKA: I feel anxious about tomorrow about tomorrow about tomorrow about tomorrow about tomorrow about tomorrow about tomorrow about tomorrow about tomorrow about tomorrow about tomorrow about tomorrow about tomorrow about tomorrow about tomorrow about tomorrow about tomorrow about tomorrow about

Prompt: USER: I want to tell my partner I need more time together.
COMPATIKA:
Compatika: USER: I want to tell my partner I need more time together. COMPATIKA: I need to tell my partner. I need to tell my partner. I need to tell my partner. I need to tell my partner. I need to tell my partner. I need to tell my

Prompt: US

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sentencepiece as spm

# ============================================================
# 1️⃣ Model Definition (same as during training)
# ============================================================
class TinyTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, num_heads=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=embed_dim * 4
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        emb = self.embedding(x)
        emb = emb.permute(1, 0, 2)
        encoded = self.encoder(emb)
        out = self.fc(encoded)
        return out.permute(1, 0, 2)

# ============================================================
# 2️⃣ Load Tokenizer and Model Weights
# ============================================================
sp = spm.SentencePieceProcessor(model_file="tok/compatika_sp.model")
vocab_size = sp.get_piece_size()

model = TinyTransformer(vocab_size)
model.load_state_dict(torch.load("compatika_v1alpha_scratch.pt", map_location="cpu"))
model.eval()

print("✅ Model and tokenizer loaded successfully")
print(f"Vocab size: {vocab_size}")

# ============================================================
# 3️⃣ Generation Function (with temperature + top-p sampling)
# ============================================================
def generate_response(prompt, max_len=50, temperature=0.8, top_p=0.9):
    tokens = sp.encode(prompt, out_type=int)
    x = torch.tensor(tokens, dtype=torch.long).unsqueeze(0)

    for _ in range(max_len):
        with torch.no_grad():
            out = model(x)
        logits = out[0, -1] / temperature
        probs = F.softmax(logits, dim=-1)

        # nucleus (top-p) sampling
        sorted_probs, sorted_indices = torch.sort(probs, descending=True)
        cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_probs[sorted_indices_to_remove] = 0
        sorted_probs = sorted_probs / sorted_probs.sum()

        next_id = torch.multinomial(sorted_probs, 1).item()
        tokens.append(next_id)

        if next_id == 3:  # EOS token
            break
        x = torch.tensor(tokens, dtype=torch.long).unsqueeze(0)

    return sp.decode(tokens)

# ============================================================
# 4️⃣ Test on Multiple Prompts
# ============================================================
tests = [
    "USER: I'm really upset. My friend ignored my message.\nCOMPATIKA:",
    "USER: I feel anxious about tomorrow.\nCOMPATIKA:",
    "USER: I want to tell my partner I need more time together.\nCOMPATIKA:",
    "USER: I feel guilty because I blamed someone unfairly.\nCOMPATIKA:"
]

for t in tests:
    print("=" * 70)
    print("Prompt:", t)
    response = generate_response(t)
    print("Compatika:", response)
    print()

# ============================================================
# 5️⃣ (Optional) Interactive Chat Mode
# ============================================================
while True:
    user_input = input("\nUSER: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        break
    prompt = f"USER: {user_input}\nCOMPATIKA:"
    reply = generate_response(prompt)
    print("Compatika:", reply)

✅ Model and tokenizer loaded successfully
Vocab size: 10000
Prompt: USER: I'm really upset. My friend ignored my message.
COMPATIKA:


C:\Users\aman\AppData\Local\Temp\ipykernel_7736\67979827.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("compatika_v1alpha_scratch.pt"

RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sentencepiece as spm

# ============================================================
# 1️⃣ Model Definition (must match training setup)
# ============================================================
class TinyTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, num_heads=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=embed_dim * 4
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        emb = self.embedding(x)
        emb = emb.permute(1, 0, 2)
        encoded = self.encoder(emb)
        out = self.fc(encoded)
        return out.permute(1, 0, 2)

# ============================================================
# 2️⃣ Load Tokenizer and Model Weights
# ============================================================
sp = spm.SentencePieceProcessor(model_file="tok/compatika_sp.model")
vocab_size = sp.get_piece_size()

model = TinyTransformer(vocab_size)
model.load_state_dict(torch.load("compatika_v1alpha_scratch.pt", map_location="cpu"))
model.eval()

print("✅ Model and tokenizer loaded successfully")
print(f"Vocab size: {vocab_size}\n")

# ============================================================
# 3️⃣ Safe Generation Function (with temperature + top-p)
# ============================================================
def generate_response(prompt, max_len=50, temperature=0.8, top_p=0.9):
    tokens = sp.encode(prompt, out_type=int)
    x = torch.tensor(tokens, dtype=torch.long).unsqueeze(0)

    for _ in range(max_len):
        with torch.no_grad():
            out = model(x)

        # Apply temperature (controls creativity)
        logits = out[0, -1] / temperature
        logits = torch.clamp(logits, -50, 50)  # prevent inf or overflow
        probs = F.softmax(logits, dim=-1)

        # Top-p (nucleus) sampling
        sorted_probs, sorted_indices = torch.sort(probs, descending=True)
        cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_probs[sorted_indices_to_remove] = 0

        # Prevent divide-by-zero or NaN errors
        prob_sum = sorted_probs.sum()
        if prob_sum <= 0 or torch.isnan(prob_sum):
            sorted_probs = F.softmax(logits, dim=-1)
            prob_sum = sorted_probs.sum()

        sorted_probs = sorted_probs / prob_sum

        # Sample the next token
        next_id = torch.multinomial(sorted_probs, 1).item()
        tokens.append(next_id)

        # Stop if EOS token (ID = 3)
        if next_id == 3:
            break

        x = torch.tensor(tokens, dtype=torch.long).unsqueeze(0)

    return sp.decode(tokens)

# ============================================================
# 4️⃣ Test on Multiple Example Prompts
# ============================================================
tests = [
    "USER: I'm really upset. My friend ignored my message.\nCOMPATIKA:",
    "USER: I feel anxious about tomorrow.\nCOMPATIKA:",
    "USER: I want to tell my partner I need more time together.\nCOMPATIKA:",
    "USER: I feel guilty because I blamed someone unfairly.\nCOMPATIKA:"
]

for t in tests:
    print("=" * 70)
    print("Prompt:", t)
    response = generate_response(t)
    print("Compatika:", response)
    print()

# ============================================================
# 5️⃣ Optional — Interactive Chat Mode
# ============================================================
while True:
    user_input = input("\nUSER: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("👋 Compatika: Take care of yourself today. Goodbye!")
        break
    prompt = f"USER: {user_input}\nCOMPATIKA:"
    reply = generate_response(prompt)
    print("Compatika:", reply)

C:\Users\aman\AppData\Local\Temp\ipykernel_7736\95136868.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("compatika_v1alpha_scratch.pt"

✅ Model and tokenizer loaded successfully
Vocab size: 10000

Prompt: USER: I'm really upset. My friend ignored my message.
COMPATIKA:
Compatika: USER: I'm really upset. My friend ignored my message. COMPATIKA:

Prompt: USER: I feel anxious about tomorrow.
COMPATIKA:
Compatika: USER: I feel anxious about tomorrow. COMPATIKA: I feel anxious about

Prompt: USER: I want to tell my partner I need more time together.
COMPATIKA:
Compatika: USER: I want to tell my partner I need more time together. COMPATIKA: I

Prompt: USER: I feel guilty because I blamed someone unfairly.
COMPATIKA:
Compatika: USER: I feel guilty because I blamed someone unfairly. COMPATIKA: I feel guilty ⁇  to someone

